In [246]:
import numpy as np
import tensorflow as tf
# import random as rn

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation, Dropout
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

In [228]:
filepath_files = '/Users/krahman/work/tutorials/tensorflow_classification/data/'

train_samples = pd.read_csv(filepath_files + 'mod_x_train.csv').drop('Unnamed: 0', axis=1)
train_labels = pd.read_csv(filepath_files + 'mod_y_train.csv').drop('Unnamed: 0', axis=1)
test_samples = pd.read_csv(filepath_files + 'mod_x_test.csv').drop('Unnamed: 0', axis=1)
test_labels = pd.read_csv(filepath_files + 'mod_y_test.csv').drop('470624', axis=1)
test_labels = test_labels.rename(columns={"0":"isFraud"})

test_samples = pd.concat([test_samples, test_labels], axis=1)
test_samples, val_samples = train_test_split(test_samples, test_size=.4)

train_samples = train_samples.reset_index(drop=True)
test_samples = test_samples.reset_index(drop=True)
val_samples = val_samples.reset_index(drop=True)

# test_samples = test_samples.drop(test_samples.tail(3).index)
# val_samples = val_samples.drop(val_samples.tail(1).index)

In [229]:
train_labels = train_labels.pop('0')
test_labels = test_samples.pop('isFraud')
val_labels = val_samples.pop('isFraud')

In [230]:
print("Training data shape:", train_samples.shape)
print("Validation data shape:", val_samples.shape)
print("Testing data shape:", test_samples.shape)

(190000, 233)
(23622, 233)
(35432, 233)


In [231]:
train_labels = np.array(train_labels.astype(int))
test_labels = np.array(test_labels.fillna(0).astype(int))
val_labels = np.array(val_labels.astype(int))

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
val_samples = np.array(val_samples)

In [232]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_samples = scaler.fit_transform(train_samples)#.reshape(-1,1))
scaled_test_samples = scaler.fit_transform(test_samples)#.reshape(-1,1))
scaled_val_samples = scaler.fit_transform(val_samples)#.reshape(-1,1))

In [241]:
##################### testing 
loss = keras.losses.BinaryCrossentropy()

In [ ]:
# testing workers, next test metrics
metrics = [keras.metrics.AUC(name='auc'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.FalseNegatives(name='fn')]

batch_size = 10
epochs = 10
lr = .0001

model = Sequential([
    Dense(16, input_shape=(233,), activation='relu'),
    Dense(1, activation='softmax')
])
model.compile(Adam(lr=lr), loss=loss, metrics=metrics)
model.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2, workers=4, use_multiprocessing=True)

predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)
rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
cm = confusion_matrix(test_labels, rounded_predictions)
cm

Train on 190000 samples, validate on 23622 samples
Epoch 1/10
 - 66s - loss: 7.6666 - auc: 0.5000 - fp: 95000.0000 - fn: 0.0000e+00 - val_loss: 14.7873 - val_auc: 0.5000 - val_fp: 22781.0000 - val_fn: 0.0000e+00
Epoch 2/10
 - 62s - loss: 7.6666 - auc: 0.5000 - fp: 95000.0000 - fn: 0.0000e+00 - val_loss: 14.7873 - val_auc: 0.5000 - val_fp: 22781.0000 - val_fn: 0.0000e+00
Epoch 3/10
 - 57s - loss: 7.6666 - auc: 0.5000 - fp: 95000.0000 - fn: 0.0000e+00 - val_loss: 14.7873 - val_auc: 0.5000 - val_fp: 22781.0000 - val_fn: 0.0000e+00
Epoch 4/10


In [ ]:
# testing workers
metrics = [keras.metrics.AUC(name='auc'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.FalseNegatives(name='fn')]

batch_size = 10
epochs = 10
lr = .0001

model = Sequential([
    Dense(16, input_shape=(233,), activation='relu'),
    Dense(1, activation='softmax')
])
model.compile(Adam(lr=lr), loss=loss, metrics=metrics)
model.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2, workers=1, use_multiprocessing=True)

predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)
rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
cm = confusion_matrix(test_labels, rounded_predictions)
cm

In [240]:
metrics = [keras.metrics.AUC(name='auc'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.FalseNegatives(name='fn')]

batch_size = 10
epochs = 3
lr = .000001
lr = .01

model3 = Sequential([
    Dense(512, input_shape=(233,), activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='softmax')
])
model3.compile(Adam(lr=lr), loss=loss, metrics=metrics)
model3.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2, workers=4, use_multiprocessing=True)

predictions = model3.predict(scaled_test_samples, batch_size=10, verbose=0)
rounded_predictions = model3.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
cm = confusion_matrix(test_labels, rounded_predictions)
cm

Train on 190000 samples, validate on 23622 samples
Epoch 1/3


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/krahman/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-240-6ea8923af3ee>", line 25, in <module>
    batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2, workers=8, use_multiprocessing=True)
  File "/Users/krahman/opt/anaconda3/lib/python3.7/site-packages/keras/engine/training.py", line 1239, in fit
    validation_freq=validation_freq)
  File "/Users/krahman/opt/anaconda3/lib/python3.7/site-packages/keras/engine/training_arrays.py", line 196, in fit_loop
    outs = fit_function(ins_batch)
  File "/Users/krahman/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/backend.py", line 3740, in __call__
    outputs = self._graph_fn(*converted_inputs)
  File "/Users/krahman/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/eager/function.py", line 1081, in __call__
    retu

KeyboardInterrupt: 

In [75]:
# testing
model3 = Sequential([
    Dense(256, input_shape=(233,), activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(2, activation='softmax')
])
model3.compile(Adam(lr=.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model3.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=10, epochs=15, shuffle=True, verbose=2, workers=8, use_multiprocessing=True)

Train on 190000 samples, validate on 11811 samples
Epoch 1/15
 - 217s - loss: 0.3056 - accuracy: 0.8685 - val_loss: 0.3789 - val_accuracy: 0.9004
Epoch 2/15
 - 215s - loss: 0.2416 - accuracy: 0.9009 - val_loss: 0.4035 - val_accuracy: 0.8996
Epoch 3/15
 - 215s - loss: 0.2200 - accuracy: 0.9112 - val_loss: 0.4145 - val_accuracy: 0.8894
Epoch 4/15
 - 215s - loss: 0.2051 - accuracy: 0.9178 - val_loss: 0.4427 - val_accuracy: 0.8854
Epoch 5/15
 - 235s - loss: 0.1941 - accuracy: 0.9224 - val_loss: 0.4560 - val_accuracy: 0.8824
Epoch 6/15
 - 215s - loss: 0.1839 - accuracy: 0.9267 - val_loss: 0.5035 - val_accuracy: 0.8733
Epoch 7/15
 - 215s - loss: 0.1762 - accuracy: 0.9307 - val_loss: 0.5650 - val_accuracy: 0.8870
Epoch 8/15
 - 216s - loss: 0.1690 - accuracy: 0.9343 - val_loss: 0.7868 - val_accuracy: 0.8779
Epoch 9/15
 - 215s - loss: 0.1631 - accuracy: 0.9365 - val_loss: 0.7460 - val_accuracy: 0.8504
Epoch 10/15
 - 216s - loss: 0.1577 - accuracy: 0.9385 - val_loss: 0.8295 - val_accuracy: 0.838

In [73]:
# BEST
model3 = Sequential([
    Dense(256, input_shape=(233,), activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(2, activation='softmax')
])
model3.compile(Adam(lr=.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model3.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=10, epochs=10, shuffle=True, verbose=2, workers=8, use_multiprocessing=True)

predictions = model3.predict(scaled_test_samples, batch_size=10, verbose=0)
rounded_predictions = model3.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
cm = confusion_matrix(test_labels, rounded_predictions)
cm

Train on 190000 samples, validate on 11811 samples
Epoch 1/10
 - 219s - loss: 0.3035 - accuracy: 0.8689 - val_loss: 0.2884 - val_accuracy: 0.9268
Epoch 2/10
 - 282s - loss: 0.2410 - accuracy: 0.9009 - val_loss: 0.3550 - val_accuracy: 0.8940
Epoch 3/10
 - 224s - loss: 0.2197 - accuracy: 0.9111 - val_loss: 0.3425 - val_accuracy: 0.9085
Epoch 4/10
 - 218s - loss: 0.2038 - accuracy: 0.9185 - val_loss: 0.3719 - val_accuracy: 0.8928
Epoch 5/10
 - 237s - loss: 0.1916 - accuracy: 0.9240 - val_loss: 0.4116 - val_accuracy: 0.8882
Epoch 6/10
 - 230s - loss: 0.1819 - accuracy: 0.9283 - val_loss: 0.4760 - val_accuracy: 0.8866
Epoch 7/10
 - 228s - loss: 0.1743 - accuracy: 0.9317 - val_loss: 0.4758 - val_accuracy: 0.8853
Epoch 8/10
 - 282s - loss: 0.1674 - accuracy: 0.9346 - val_loss: 0.4777 - val_accuracy: 0.8734
Epoch 9/10
 - 255s - loss: 0.1611 - accuracy: 0.9377 - val_loss: 0.5180 - val_accuracy: 0.8962
Epoch 10/10
 - 264s - loss: 0.1564 - accuracy: 0.9396 - val_loss: 0.5364 - val_accuracy: 0.883

array([[41558,  4014],
       [ 1520,   151]])

In [74]:
# BEST
model3 = Sequential([
    Dense(256, input_shape=(233,), activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(2, activation='softmax')
])
model3.compile(Adam(lr=.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model3.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=10, epochs=10, shuffle=True, verbose=2, workers=8, use_multiprocessing=True)

predictions = model3.predict(scaled_test_samples, batch_size=10, verbose=0)
rounded_predictions = model3.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
cm = confusion_matrix(test_labels, rounded_predictions)
cm

Train on 190000 samples, validate on 11811 samples
Epoch 1/10
 - 271s - loss: 0.3938 - accuracy: 0.8214 - val_loss: 0.3914 - val_accuracy: 0.8572
Epoch 2/10
 - 265s - loss: 0.2804 - accuracy: 0.8833 - val_loss: 0.4753 - val_accuracy: 0.8328
Epoch 3/10
 - 274s - loss: 0.2562 - accuracy: 0.8957 - val_loss: 0.3714 - val_accuracy: 0.8996
Epoch 4/10
 - 280s - loss: 0.2413 - accuracy: 0.9030 - val_loss: 0.3650 - val_accuracy: 0.9076
Epoch 5/10
 - 266s - loss: 0.2311 - accuracy: 0.9076 - val_loss: 0.3622 - val_accuracy: 0.9145
Epoch 6/10
 - 332s - loss: 0.2235 - accuracy: 0.9109 - val_loss: 0.3747 - val_accuracy: 0.9116
Epoch 7/10
 - 303s - loss: 0.2167 - accuracy: 0.9141 - val_loss: 0.3681 - val_accuracy: 0.9222
Epoch 8/10
 - 305s - loss: 0.2111 - accuracy: 0.9169 - val_loss: 0.3811 - val_accuracy: 0.9230
Epoch 9/10
 - 303s - loss: 0.2066 - accuracy: 0.9184 - val_loss: 0.3763 - val_accuracy: 0.9257
Epoch 10/10
 - 295s - loss: 0.2015 - accuracy: 0.9211 - val_loss: 0.3689 - val_accuracy: 0.927

array([[42868,  2704],
       [ 1565,   106]])

In [46]:
# BEST testing .0001
model7 = Sequential([
    Dense(256, input_shape=(233,), activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(2, activation='softmax')
])
model7.compile(Adam(lr=.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model7.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=10, epochs=10, shuffle=True, verbose=2, workers=4, use_multiprocessing=True)

Train on 190000 samples, validate on 11811 samples
Epoch 1/10
 - 205s - loss: 0.3026 - accuracy: 0.8688 - val_loss: 0.9822 - val_accuracy: 0.6884
Epoch 2/10
 - 212s - loss: 0.2402 - accuracy: 0.9015 - val_loss: 1.2454 - val_accuracy: 0.5444
Epoch 3/10
 - 196s - loss: 0.2213 - accuracy: 0.9104 - val_loss: 0.7317 - val_accuracy: 0.7308
Epoch 4/10
 - 206s - loss: 0.2070 - accuracy: 0.9167 - val_loss: 1.1737 - val_accuracy: 0.7072
Epoch 5/10
 - 204s - loss: 0.1956 - accuracy: 0.9220 - val_loss: 0.7927 - val_accuracy: 0.7717
Epoch 6/10
 - 194s - loss: 0.1857 - accuracy: 0.9267 - val_loss: 1.1894 - val_accuracy: 0.6619
Epoch 7/10
 - 202s - loss: 0.1774 - accuracy: 0.9301 - val_loss: 0.9937 - val_accuracy: 0.7533
Epoch 8/10
 - 202s - loss: 0.1699 - accuracy: 0.9330 - val_loss: 1.3999 - val_accuracy: 0.7167
Epoch 9/10
 - 194s - loss: 0.1640 - accuracy: 0.9350 - val_loss: 0.7933 - val_accuracy: 0.8282
Epoch 10/10
 - 204s - loss: 0.1585 - accuracy: 0.9381 - val_loss: 1.0263 - val_accuracy: 0.819